In [43]:
import pandas as pd
import numpy as np
import datetime
import glob
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

In [3]:
df = pd.read_csv('./data/train.csv', encoding='windows-1251')

/home/fisher/anaconda3/envs/ml/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df.head()

,Interval,Date,OrderDate,ClientID,ChannelID,OrderID,MaterialID,GroupID,Cluster,CancelFlag,OrderCnt,DeliveryType,prepay,count_edit
0,14-16.,03/10/2018,02/10/2018,93808186,2,90102063002,3328810.0,61.0,NaN,0,1.0,Обычная доставка,0,1
1,14-16.,03/10/2018,02/10/2018,93808186,2,90102063002,3281258.0,30.0,NaN,0,2.0,Обычная доставка,0,1
2,14-16.,03/10/2018,02/10/2018,93808186,2,90102063002,3210734.0,10.0,NaN,0,1.0,Обычная доставка,0,1
3,14-16.,03/10/2018,02/10/2018,93808186,2,90102063002,3328848.0,61.0,NaN,0,2.0,Обычная доставка,0,1
4,12-14.,07/10/2018,05/10/2018,94112406,2,90102091007,3347801.0,17.0,NaN,0,10.0,Обычная доставка,0,1


In [5]:
df["Cluster"] = df["Cluster"].fillna('-1')
groups_counts = list(df['GroupID'].value_counts()[df['GroupID'].value_counts() > 10000].keys())
df['GroupID'] = df['GroupID'].apply(lambda x: x if x in groups_counts else -1)
materials_counts = list(df['MaterialID'].value_counts()[df['MaterialID'].value_counts() > 10000].keys())
df['MaterialID'] = df['MaterialID'].apply(lambda x: x if x in materials_counts else -1)

In [6]:
cols = ['OrderID', 'Interval', 'Date', 'OrderDate', 'ClientID', 'ChannelID','CancelFlag',
       'Cluster', 'DeliveryType', 'prepay', 'count_edit']

In [29]:
df_ = df.groupby(cols).sum().reset_index()[['OrderID', 'Interval', 'Date', 'OrderDate', 'ClientID', 'ChannelID','CancelFlag',
       'Cluster', 'DeliveryType', 'prepay', 'count_edit', 'OrderCnt']]

In [8]:
orders = df_['OrderID']

In [9]:
df_dummies = pd.get_dummies(df[df['OrderID'].isin(orders)][['OrderID', 'MaterialID', 'GroupID']], columns=['MaterialID', 'GroupID'])

In [15]:
df_group1 = df_dummies.head(5000000).groupby('OrderID').sum()
df_group2 = df_dummies.tail(len(df_dummies) - 5000000 - 1).groupby('OrderID').sum()

In [25]:
df_group = df_group1.append(df_group2)

In [30]:
df_ = df_.merge(df_group, on='OrderID', right_index=True)

In [32]:
df_['OrderDate_weekday'] = df_['OrderDate'].apply(lambda x: datetime.datetime.strptime(x, '%d/%m/%Y').weekday())
df_['Date_weekday'] = df_['Date'].apply(lambda x: datetime.datetime.strptime(x, '%d/%m/%Y').weekday())

In [33]:
def to_labels(series):
    le = LabelEncoder()
    return le.fit_transform(series)

In [38]:
cols = ['Interval', 'ChannelID', 'DeliveryType', 'Cluster']

for col in cols:
    df_[col] = to_labels(df_[col])

In [39]:
df_

,OrderID,Interval,Date,OrderDate,ClientID,ChannelID,CancelFlag,Cluster,DeliveryType,prepay,...,GroupID_41.0,GroupID_42.0,GroupID_46.0,GroupID_55.0,GroupID_59.0,GroupID_61.0,GroupID_63.0,GroupID_66.0,OrderDate_weekday,Date_weekday
0,90102063002,7,03/10/2018,02/10/2018,93808186,0,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,1,2
1,90102091007,5,07/10/2018,05/10/2018,94112406,0,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,4,6
2,90102092000,14,05/10/2018,05/10/2018,93696397,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,4
3,90102103017,20,12/10/2018,12/10/2018,93696397,0,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,4,4
4,90102104012,4,17/10/2018,15/10/2018,93411902,0,1,0,1,0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354846,98357278690,7,31/12/2018,30/12/2018,93324410,0,0,18,1,0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,6,0
354847,98357278797,20,30/12/2018,30/12/2018,94273838,4,0,18,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6,6
354848,98357279051,7,31/12/2018,30/12/2018,93899170,0,0,18,1,0,...,5.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,6,0
354849,98357281094,25,31/12/2018,30/12/2018,91690087,0,0,18,1,0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,6,0


In [40]:
df_res = df_.drop(['OrderID', 'Date', 'OrderDate', 'ClientID'], axis=1)

In [42]:
df_res.to_csv('./data/data_pre.csv', index=False)